In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Age option 2 APOKASC-2 Pinsonneault et al. 2018
#Reading in the table, making sure all the tables have a column named Age in Gyr
# and that every star in the table has an Age
apokasc2raw = Table.read("Pinsonneault2018.txt", format="ascii.cds")
apokasc2raw['Age']=(10**np.array(apokasc2raw['LogAge'])/1000) #Age was in log(Myr) so needs converting
hasagea2=np.where((apokasc2raw['Age']==apokasc2raw['Age']) & (apokasc2raw['Age']>0.1))
agedata=apokasc2raw[hasagea2]

In [2]:
from tensorflow import keras

In [3]:
filename='astraAllStarASPCAP-0.6.0.fits'
tb = fits.open(filename)
header=tb[2].header
data = tb[2].data 

mask_gaia = (data['zgr_plx']>0)
data_masked=data[mask_gaia]

intersect, ind_a, ind_b = np.intersect1d(data_masked['sdss4_apogee_id'],agedata['2MASS'], return_indices=True) 

fullx = np.dstack([data_masked['teff'][ind_a],data_masked['logg'][ind_a], data_masked['m_h_atm'][ind_a],
                   data_masked['alpha_m_atm'][ind_a], data_masked['c_h'][ind_a], data_masked['n_h'][ind_a]])[0]

fully = np.dstack([agedata['Age'][ind_b]])[0] #for Pinsonneault 2018

#remove non-finite entries!
mask = np.all(np.isfinite(fullx), axis=1) & np.all(np.isfinite(fully), axis=1)
fullx, fully = fullx[mask], fully[mask]

scaling_x = np.median(fullx, axis=0)
scaling_y = np.median(fully, axis=0)

fullx, fully = fullx/scaling_x, fully/scaling_y

In [4]:
neurons_per_layer=50
layers=6
iterations=50

In [5]:

inputs = keras.Input(shape=(6,))

layer1 = keras.layers.Dense(neurons_per_layer, activation='relu')(inputs)
layer2 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer1)
layer3 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer2)
layer4 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer3)
layer5 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer4)
layer6 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer5)

outputs = keras.layers.Dense(1)(layer6)

model = keras.Model(inputs=inputs, outputs=outputs, name='test')

model.summary()

Model: "test"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 6)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 50)                  │             350 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 50)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 50)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 50)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 50)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 50)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,151 (51.37 KB)

 Trainable params: 13,151 (51.37 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

tenpercent=len(agedata['Age'][ind_b])//10 

trainbin=slice(0,-1*tenpercent-1)
testing=slice(-1*tenpercent,-1)

x_train, y_train = fullx[trainbin], fully[trainbin]
x_test, y_test = fullx[testing], fully[testing]

In [7]:
model.fit(x_train, y_train, epochs=iterations, validation_split=0.05, batch_size=300)

Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.0011 - loss: 0.7986 - val_accuracy: 0.0000e+00 - val_loss: 0.4762
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0013 - loss: 0.4380 - val_accuracy: 0.0000e+00 - val_loss: 0.3272
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0013 - loss: 0.3539 - val_accuracy: 0.0000e+00 - val_loss: 0.2737
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0013 - loss: 0.2932 - val_accuracy: 0.0000e+00 - val_loss: 0.2424
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0013 - loss: 0.2581 - val_accuracy: 0.0000e+00 - val_loss: 0.2233
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0013 - loss: 0.2514 - val_accuracy: 0.0000e+00 - val_loss: 0.2247
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0013 - loss: 0.2418 - val_accuracy: 0.0000e+00 - val_loss: 0.2255
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0013 - loss: 0.2377 - va

In [8]:
predictions = model.predict(x_test)
print(len(predictions))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
651


In [9]:
neurons_per_layer_model2=500
layers_model2=14
iterations_model2=200

In [10]:
inputs2 = keras.Input(shape=(6,))

layer1model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(inputs2)
layer2model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer1model2)
layer3model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer2model2)
layer4model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer3model2)
layer5model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer4model2)
layer6model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer5model2)
layer7model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer6model2)
layer8model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer7model2)
layer9model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer8model2)
layer10model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer9model2)
layer11model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer10model2)
layer12model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer11model2)
layer13model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer12model2)
layer14model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer13model2)

outputs2 = keras.layers.Dense(1)(layer14model2)

model2 = keras.Model(inputs=inputs2, outputs=outputs2, name='second_model')

model2.summary()

Model: "second_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 6)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 500)                 │           3,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 1)                   │             501 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,260,501 (12.44 MB)

 Trainable params: 3,260,501 (12.44 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model2.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model2.fit(x_train, y_train, epochs=iterations_model2, validation_split=0.05, batch_size=300)

Epoch 1/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - accuracy: 1.8625e-04 - loss: 1.8959 - val_accuracy: 0.0000e+00 - val_loss: 0.4784
Epoch 2/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.0013 - loss: 0.4104 - val_accuracy: 0.0000e+00 - val_loss: 0.3108
Epoch 3/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.0013 - loss: 0.3138 - val_accuracy: 0.0000e+00 - val_loss: 0.2774
Epoch 4/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.0013 - loss: 0.3033 - val_accuracy: 0.0000e+00 - val_loss: 0.2454
Epoch 5/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.0013 - loss: 0.2614 - val_accuracy: 0.0000e+00 - val_loss: 0.2335
Epoch 6/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.0013 - loss: 0.2468 - val_accuracy: 0.0000e+00 - val_loss: 0.2246
Epoch 7/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.0013 - loss: 0.2444 - val_accuracy: 0.0000e+00 - val_loss: 0.2286
Epoch 8/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.0013 -

In [12]:
predictions2 = model2.predict(x_test)
print(len(predictions2))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
651


In [13]:
metric=0.3 #is the accuracy better than 30%?
goodfit=np.where(((1-metric) < predictions/y_test) & ((1+metric) > predictions/y_test)) 
badfit=np.where(((1-metric) > predictions/y_test) | ((1+metric) < predictions/y_test))

print ('With ', neurons_per_layer, 'neurons per layer, ', layers, 'layers, and ', iterations, 'iterations')
print ('using the training set', trainbin)
print (len(goodfit[0])/len(y_test)*100, 'percent of the ages are good')
print (len(badfit[0])/len(y_test)*100, 'percent of the ages are bad')

With  50 neurons per layer,  6 layers, and  50 iterations
using the training set slice(0, -653, None)
59.44700460829493 percent of the ages are good
40.55299539170507 percent of the ages are bad


In [14]:
goodfit2=np.where(((1-metric) < predictions2/y_test) & ((1+metric) > predictions2/y_test)) 
badfit2=np.where(((1-metric) > predictions2/y_test) | ((1+metric) < predictions2/y_test))

print ('With ', neurons_per_layer_model2, 'neurons per layer, ', layers_model2, 'layers, and ', iterations_model2, 'iterations')
print ('using the training set', trainbin)
print (len(goodfit2[0])/len(y_test)*100, 'percent of the ages are good')
print (len(badfit2[0])/len(y_test)*100, 'percent of the ages are bad')

With  500 neurons per layer,  14 layers, and  200 iterations
using the training set slice(0, -653, None)
63.594470046082954 percent of the ages are good
36.405529953917046 percent of the ages are bad


In [16]:
difference = abs(predictions-predictions2)
print(difference)

from astropy.table import QTable
import astropy.units as u

totaldata = Table([predictions, predictions2, difference], names=('Model 1 Predictions (Gyr)','Model 2 Predictions (Gyr)','Difference (Gyr)'), meta={'name': 'first table'})
print(totaldata)

print("The average difference is",np.mean(difference),"Gyr")

[[1.57351673e-01]
 [5.43835163e-02]
 [5.48710823e-02]
 [1.67608336e-01]
 [4.31585312e-03]
 [2.21268177e-01]
 [4.77852821e-02]
 [5.67216277e-02]
 [3.92942727e-02]
 [1.34111434e-01]
 [1.48059607e-01]
 [3.72391045e-02]
 [5.11171967e-02]
 [7.79032707e-02]
 [4.71257210e-01]
 [1.43152058e-01]
 [2.51382589e-02]
 [6.60184026e-02]
 [4.83651161e-02]
 [2.37016916e-01]
 [3.58470917e-01]
 [8.06717873e-02]
 [2.74724811e-02]
 [1.02823675e-02]
 [1.51870728e-01]
 [5.59806824e-02]
 [1.04424238e-01]
 [9.61482525e-03]
 [2.18490362e-02]
 [3.46728683e-01]
 [6.87650740e-02]
 [1.05792239e-01]
 [7.21498132e-02]
 [9.75841284e-02]
 [6.13989234e-02]
 [4.71529663e-02]
 [4.54032421e-03]
 [6.83274269e-02]
 [1.70287967e-01]
 [1.14446282e-02]
 [8.81364346e-02]
 [1.21984959e-01]
 [5.00891805e-02]
 [1.60162628e-01]
 [5.38836122e-02]
 [1.03650689e-02]
 [1.66732967e-01]
 [8.46131444e-02]
 [2.68144011e-02]
 [1.52167499e-01]
 [2.88112283e-01]
 [1.43278897e-01]
 [6.42606378e-01]
 [1.16779208e-01]
 [1.39974296e-01]
 [1.969566

In [18]:
totaldata.write("APOKASC2 trained on 2 different neural models.ecsv")